# VISION PIPELINE

## Download/Load Data

In [ ]:
import fastai
print(f"FastAi: {fastai.__version__}")

from fastai.vision.all import * # Import all modules and functions from Fastai's vision package

In [ ]:
from fastai.data.transforms import get_image_files

path = untar_data(URLs.PETS)
fnames = get_image_files(path = path/'images')
print(f"Total {len(fnames)} images, {fnames[:10]} ...")

## Data Preparation

In [ ]:
# Data Transformation
from fastai.vision.augment import aug_transforms, RandomResizedCrop
from fastai.vision.core import imagenet_stats

item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=224, max_warp=0),Normalize.from_stats(*imagenet_stats)]

## DataLoader

In [ ]:
### DataLoader
from fastai.vision.data import ImageBlock
from fastai.data.block import CategoryBlock, DataBlock
from fastai.data.transforms import get_image_files, Normalize, RandomSplitter, RegexLabeller

pets = DataBlock(blocks=(ImageBlock, CategoryBlock),
                 get_items=get_image_files,
                 splitter=RandomSplitter(),
                 get_y=RegexLabeller(pat=r'/([^/]+)_\d+.*'),
                 item_tfms=item_tfms,
                 batch_tfms=batch_tfms)

dls = pets.dataloaders(source=path/'images', bs=64)
dls.show_batch() # visualize the preprocessed image
print(f"{len(dls.vocab)} {dls.vocab}")

## Define Model & Find Learning Rate

In [ ]:
from fastai.callback.fp16 import to_fp16 # Mixed Precision training
from fastai.vision.all import vision_learner
from torchvision.models.resnet import resnet50 
from fastai.metrics import error_rate

### Model/Learner
learn = vision_learner(dls=dls, 
                       arch=resnet50, 
                       pretrained=True, 
                       metrics=error_rate).to_fp16()

# Find Learning Rate
learn.lr_find()

## Model Training

In [ ]:
lr = slice(1e-4,1e-2)

### Model Training
learn.fit_one_cycle(4, lr_max=lr)

# Save Model
learn.save('model1_freezed')

## Model Interpretation

In [ ]:
# Model Interpretation
from fastai.interpret import ClassificationInterpretation

interp = ClassificationInterpretation.from_learner(learn)
losses,idxs = interp.top_losses()

print(len(dls.valid_ds)==len(losses)==len(idxs)) # True

interp.plot_top_losses(k=9, figsize=(15,10)) # pred/actual/loss/prob

In [ ]:
interp.plot_confusion_matrix(figsize=(16,16), dpi=60)

## Unfreeze Model, Fine Tune, Learning Rate

In [ ]:
### Unfreezing our Model, fine-tuning, and our learning rates
learn.load('model1_freezed')
learn.lr_find()

In [ ]:
lr = slice(1e-5,1e-4)

learn.unfreeze()

learn.fit_one_cycle(4,lr_max=lr)

learn.save('model2_unfreezed')

## Inference

In [ ]:
!wget -O pug.jpg "https://th.bing.com/th/id/OIP.t7e7M-C6RM8T9a5xMgBsYgHaE8?w=273&h=182&c=7&r=0&o=5&dpr=2&pid=1.7"

In [ ]:
!tree /kaggle

In [ ]:
# Load the trained model
learn.load('model2_unfreezed')

In [ ]:
from PIL import Image
# Prepare the input image (you should replace 'image_path' with the path to your image)
input_image = Image.open('/kaggle/working/pug.jpg')
input_image

In [ ]:
# Perform inference
prediction = learn.predict(input_image)

# Display the results
print("Predicted Class:", prediction[0])
print("Predicted Probabilities:", prediction[2])

In [ ]:
prediction

In [ ]:
dls.vocab[29]